In [1]:
import pandas as pd

## crop

In [2]:
tr_sessions = pd.read_csv('./train_sessions_sorted.csv')
tr_sessions['date'] = pd.to_datetime(tr_sessions['date'], format='%Y-%m-%d %H:%M:%S')
tr_purchases = pd.read_csv('./train_purchases.csv')

print(tr_sessions.head())

tr_sessions['hour'] = tr_sessions['date'].dt.hour
print(tr_sessions.columns)

   session_id  Unnamed: 1  session_id.1  item_id                    date
0           3           1             3     9655 2020-12-18 21:19:48.093
1           3           0             3     9655 2020-12-18 21:25:00.373
2          13           2            13    15654 2020-03-13 19:35:27.136
3          18           5            18     4026 2020-08-26 19:15:47.232
4          18           4            18     2507 2020-08-26 19:16:31.211
Index(['session_id', 'Unnamed: 1', 'session_id.1', 'item_id', 'date', 'hour'], dtype='object')


In [1]:
## Generate Train dataset txt file
sids = tr_purchases['session_id'].unique()

with open('./train_gap1h.txt', 'w') as f:
  for sid in sids:
    ses_per_sid = tr_sessions.loc[tr_sessions['session_id'] == sid]
    ses_gr = ses_per_sid.groupby(['hour'])
    for gn, gr in ses_gr:
      sess = gr['item_id'].tolist()
      j = 0
      while j+10 < len(sess):
        sess_part = sess[j:j+10]
        sess_part.append(tr_purchases.loc[tr_purchases['session_id'] == sid]['item_id'].values[0])
        string = ",".join(map(str,sess_part)) + '\n'
        f.write(string)
        j += 10
      sess_part = sess[j:]
      sess_part.append(tr_purchases.loc[tr_purchases['session_id'] == sid]['item_id'].values[0])
      string = ",".join(map(str,sess_part)) + '\n'
      f.write(string)

NameError: name 'tr_purchases' is not defined

In [2]:
## check statistics
def print_statistics(fn):
  with open(fn, 'r') as f:
    session = f.readlines()

  len(session)
  session = [s.split(',') for s in session]
  li = list(map(len, session))
  print("average length ", sum(li)/len(li))
  print("maximum length", max(li))

  larger = sum(map((lambda x: x>5), li))
  print("session length larger than 5: ", larger)
  smaller = sum(map((lambda x: x<=5), li))
  print("session length smaller or equal to 5: ", smaller)

print_statistics('./train_gap1h.txt')

average length  4.530623591027529
maximum length 11
session length larger than 5:  374847
session length smaller or equal to 5:  968774
